<img align="center" src="figures/course.png" width="800">

#                                    16720 (B) Bag of Visual Words - Assignment 2

     Instructor: Kris Kitani                       TAs:Paritosh (Lead), Rawal, Yan, Zen, Wen-Hsuan, Qichen

In [1]:
import nbimporter
import numpy as np
import scipy.ndimage
from skimage import io
import skimage.transform
import os,time
import util
import multiprocessing
import threading
import queue
import torch
import torchvision
import torchvision.transforms

## For Autograding P4, ensure uploading `trained_conf_matrix.npy` and `trained_system_deep.npz`.

### Deep Learning Features - An Alternative to ``Bag of Words``

As we have discussed in class, another powerful method for scene classification in computer vision is the employment of convolutional neural networks (CNNs) - sometimes referred to generically as $deep learning$. It is important to understand how previously trained (pretrained) networks can be used as another form of feature extraction, and how they relate to classical Bag of Words (BoW) features. We will be covering details on how one chooses the network architecture and training procedures later in the course. For this question, however, we will be asking you to deal with the VGG-16 pretrained network. VGG-16 is a pretrained Convolutional Neural Network (CNN) that has been trained on approximately 1.2 million images from the ImageNet Dataset (``http://image-net.org/index``) by the Visual Geometry Group (VGG) at University of Oxford. The model can classify images into a 1000 object categories (e.g. keyboard, mouse, coffee mug, pencil).

One lesson we want you to take away from this exercise is to understand the effectiveness of $deep$ $features$ for general classification tasks within computer vision - even when those features have been previously trained on a different dataset (i.e. ImageNet) and task (i.e. object recognition). 

#### Extracting Deep Features

To complete this question, you need to install the ``torchvision`` library from Pytorch, a popular Python-based deep learning library.
If you are using the Anaconda package manager (``https://www.anaconda.com/download/``), this can be done with the following command:
```
            conda install pytorch torchvision -c pytorch
```
To check that you have installed it correctly, make sure that you can ``import torch`` in a Python interpreter without errors.
Please refer to ``https://pytorch.org/`` for more detailed installation instructions. 

#### Q4.1.1 (25 Points)

We want to extract out deep features corresponding to the convolutional layers of theVGG-16 network.  In this problem, we will use the trained weights from the VGG network directly, but implement our own operations. To load the network, use the line
```
        vgg16 = torchvision.models.vgg16(pretrained=True).double()
```
followed by ``vgg16.eval()``
The latter line ensures that the VGG-16 network is in evaluation mode, not training mode.

path_img = "./data/aquarium/sun_aztvjgubyrgvirup.jpg"

image = io.imread(path_img)

image = image.astype('float')/255

vgg16 = torchvision.models.vgg16(pretrained=True).double()

vgg16.eval()

We want you to complete a function that is able to output $VGG-16$ network outputs at the **fc7** layer in
```
    def extract_deep_feature(x,vgg16_weights):
```
where ``x`` refers to the input image and ``vgg16_weights`` is a structure containing the CNN's network parameters. In this function you will need to write sub-functions ``multichannel_conv2d``, ``relu``, ``max_pool2d``, and ``linear`` corresponding to the fundamental elements of the CNN: multi-channel convolution, rectified linear units (ReLU), max pooling, and fully-connected weighting.

We have provided a helper function ``util.get_VGG16_weights()`` that extracts the weight parameters of VGG-16 and its meta information. The returned variable is a numpy array of shape $L\times 3$, where $L$ is the number of layers in VGG-16. The first column of each row is a string indicating the layer type. The second/third columns may contain the learned weights and biases, or other meta-information (\eg kernel size of max-pooling). Please refer to the function docstring for details.

In order to build the ``extract_deep_feature`` function, you should run a for-loop through each layer index until layer **fc7**, which corresponds to **the second linear layer** (Refer to VGG structure to see where **fc7** is). **Remember**: the output of the preceding layer should be passed through as an input to the next.

Details on the sub-functions needed for the ``extract_deep_feature`` function can be found below.

Please use ``scipy.ndimage.convolve`` and ``numpy`` functions to implement these functions instead of using pytorch. Please keep speed in mind when implementing your function, for example, using double for loop over pixels is not a good idea.

``multichannel_conv2d(x,weight,bias)``:

a function that will perform multi-channel 2D convolution which can be defined as follows, 

\begin{equation}
\mathbf{y}^{(j)} = \sum_{k=1}^{K} \begin{bmatrix} \mathbf{x}^{(k)} * \mathbf{h}^{(j,k)} \end{bmatrix} + \mathbf{b}[j] 
\end{equation}

where $*$ denotes $2D$ convolution, $\mathbf{x} = \{ \mathbf{x}^{(k)} \}_{k=1}^{K}$ is our vectorized $K$-channel input signal, $\mathbf{h} = \{ \mathbf{h}^{(j,k)} \}_{k=1,j=1}^{K,J}$ is our $J \times K$ set of vectorized convolutional filters and $\mathbf{r} = \{ \mathbf{y}^{(j)} \}_{j=1}^{J}$ is our $J$ channel vectorized output response. Further, unlike traditional single-channel convolution CNNs often append a bias vector $\mathbf{b}$ whose $J$ elements are added to the $J$ channels of the output response. 

To implement ``multichannel_conv2d``, you can use the Scipy convolution function directly with for loops to cycle through the filters and channels (``scipy.ndimage.convolve()``). All the necessary details concerning the number of filters ($J$), number of channels ($K$), filter weights ($\mathbf{h}$) and biases ($\mathbf{b}$) can be inferred from the shapes/dimensions of the weights and biases. Notice that pytorch's convolution function actually does correlation, so to get similar answer as pytroch with scipy, you need to flip the kernel on both axes using ``np.flip()``.

In [2]:
def multichannel_conv2d(x,weight,bias):
    '''
    Performs multi-channel 2D convolution.

    [input]
    * x: numpy.ndarray of shape (H,W,input_dim)
    * weight: numpy.ndarray of shape (output_dim,input_dim,kernel_size,kernel_size)
    * bias: numpy.ndarray of shape (output_dim)

    [output]
    * feat: numpy.ndarray of shape (H,W,output_dim)
    '''
    h, w, input_dims = x.shape
    output_dims = weight.shape[0]
    final_res = np.zeros((h, w, output_dims))
    '''
    HINTS:
    1.> for 2D convolution we need to use np.fliplr and np.flipud
    2.> can use scipy.ndimage.convolve with the flipped kernel
    3.> don't forget to add the bias
    '''
    # YOUR CODE HERE
    kernel_size = weight.shape[2]
    output = []
    for i, output_channel_filter in enumerate(weight):
        accumulate = scipy.ndimage.convolve(x[:, :, 0], np.zeros((kernel_size, kernel_size)), mode='constant')
        for j, channel_filter in enumerate(output_channel_filter):
            #print(i, j)
            #print("channel image\n{}".format(x[:, :, j]))
            #print("channel filter\n{}".format(channel_filter))
            accumulate += scipy.ndimage.convolve(x[:, :, j], np.flipud(np.fliplr(channel_filter)), mode='constant')
        accumulate += bias[i]
        output.append(accumulate)
    final_res = np.stack(output, axis=-1)
    return final_res

#x = np.arange(0, 48).reshape((4, 4, 3))
#filters = [np.ones((3, 3, 3)), np.concatenate([np.ones((1, 3, 3)), np.ones((2, 3, 3)) * 2])]
#weights = np.stack(filters)
#bias = np.array([0, 1])
#print(x)
#print(bias)
#print(weights)
#print(multichannel_conv2d(x, weights, bias))

```relu(x)```:

a function that shall perform the Rectified Linear Unit (ReLU) which can be defined mathematically as,

\begin{equation}
\mbox{ReLU}(x) = \max_{x}(x, 0)
\end{equation}

and is applied independently to each element of the matrix/vector $x$ passed to it.

In [3]:
def relu(x):
    '''
    Rectified linear unit.

    [input]
    * x: numpy.ndarray

    [output]
    * y: numpy.ndarray
    '''
    # YOUR CODE HERE
    y = np.maximum(x, 0)
    return y


``max_pool2d(x,size):``

a function that shall perform max pooling over $x$ using a receptive field of size $size$ $\times$ $size$ (we assume a square receptive field here for simplicity).

  If the function receives a multi-channel input, then it should apply the max pooling operation across each input channel independently.
  
(Hint: making use of smart array indexing can drastically speed up the code.)

In [4]:
def max_pool2d(x,size):
    '''
    2D max pooling operation.

    [input]
    * x: numpy.ndarray of shape (H,W,input_dim)
    * size: pooling receptive field

    [output]
    * y: numpy.ndarray of shape (H/size,W/size,input_dim)
    '''
    h, w, dims = x.shape
    '''
    HINTS:
    1.> estimate the shape you need to apply the pooling operation.
    2.> We can smart fill the padding with np.nan and then use np.nanmax to select the max (avoiding nan)
    3.> We can input the grid (start_x:end_x, start_y:end_y, dim) as smart array indexing to np.nanmax
    '''
    # YOUR CODE HERE
    padded_x = np.pad(x, pad_width=((0, h % size), (0, w % size), (0, 0)), constant_values=np.nan)
    h_pad, w_pad, _ = padded_x.shape
    output = []
    for d in range(dims):
        channel_x = padded_x[:, :, d]
        #print("channel image\n{}".format(channel_x))
        channel_out = np.zeros((h_pad // size, w_pad // size))
        for i in range(h_pad // size):
            for j in range(w_pad // size):
                channel_out[i, j] = np.nanmax(channel_x[i * size : (i + 1) * size, j * size : (j + 1) * size])
        #print("channel out\n{}".format(channel_out))
        output.append(channel_out)
    pooled_arr = np.stack(output, axis=-1)
    return pooled_arr

#x = np.arange(0, 60).reshape((5, 4, 3))
#print(x)
#print(max_pool2d(x, 2))

``linear(x,W,b):``

a function that will compute a node vector where each element is a linear combination of the input nodes, written as

\begin{equation}
\mathbf{y}[j] = \sum_{k=1}^{K}\mathbf{W}[j,k] \mathbf{x}[k] + \mathbf{b}[j] 
\end{equation}

or more succinctly in vector form as $\mathbf{y} = \mathbf{W} \mathbf{x} + \mathbf{b}$ - where $\mathbf{x}$ is the $(K \times 1)$ input node vector, $\mathbf{W}$ is the $(J \times K)$ weight matrix, $\mathbf{b}$ is the $(J \times 1)$ bias vector and $\mathbf{y}$ is the~$(J \times 1)$ output node vector.

You should not need for-loops to implement this function.

In [5]:
def linear(x,W,b):
    '''
    Fully-connected layer.

    [input]
    * x: numpy.ndarray of shape (input_dim)
    * weight: numpy.ndarray of shape (output_dim,input_dim)
    * bias: numpy.ndarray of shape (output_dim)

    [output]
    * y: numpy.ndarray of shape (output_dim)
    '''
    
    # YOUR CODE HERE
    y = W @ x + b
    return y

You should ignore all ``DropoutLayer`` you encounter; they're functional only during the training phase.

VGG-16 assumes that all input imagery to the network is resized to $224 \times 224$ with the three color channels preserved (use ``skimage.transform.resize()`` to do this before passing any imagery to the network). And be sure to normalize the image using suggested mean and std before extracting the feature:
```
                                        mean=[0.485,0.456,0.406]}
                                        std=[0.229,0.224,0.225]}
```

In [6]:
def preprocess_image(image):
    '''
    Preprocesses the image to load into the prebuilt network.

    [input]
    * image: numpy.ndarray of shape (H,W,3)

    [output]
    * image_processed: torch.array of shape (3,H,W)
    '''

    # ----- TODO -----
    
    if(len(image.shape) == 2):
        image = np.stack((image, image, image), axis=-1)

    if(image.shape == 3 and image.shape[2] == 1):
        image = np.concatenate((image, image, image), axis=-1)

    if(image.shape[2] == 4):
        image = image[:, :, 0:3]
    '''
    HINTS:
    1.> Resize the image (look into skimage.transform.resize)
    2.> normalize the image
    3.> convert the image from numpy to torch
    '''
    # YOUR CODE HERE
    image = skimage.transform.resize(image, (224, 224))
    output = []
    dim = 3
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    for d in range(dim):
        channel_image = image[:, :, d]
        normalized_image = channel_image / np.std(channel_image) * stds[d]
        normalized_image = normalized_image - np.mean(normalized_image) + means[d]
        output.append(normalized_image)
        
    image_processed = torch.tensor(np.stack(output, axis=0))
    return image_processed

For efficiency you should check that each sub-function is working properly before putting them all together - otherwise it will be hard to track any errors. To compare your implementation with pytroch, you should compare the extracted features between your ``extract_deep_feature``  and the pre-trained VGG-16 network.  ``evaluate_deep_extractor`` should come in handy in comparing the result of the two extracted features.

In [12]:
def extract_deep_feature(x, vgg16_weights):
    '''
    Extracts deep features from the given VGG-16 weights.

    [input]
    * x: numpy.ndarray of shape (H,W,3)
    * vgg16_weights: list of shape (L,3)

    [output]
    * feat: numpy.ndarray of shape (K)
    '''
    
    feat = x.copy()
    print(x.shape)
    
    # YOUR CODE HERE
    for i, layer_data in enumerate(vgg16_weights[0:34]):
        layer_type = layer_data[0]
        print(i, layer_type, feat.shape)
        if layer_type == "conv2d":
            weights = layer_data[1]
            bias = layer_data[2]
            feat = multichannel_conv2d(feat, weights, bias)
        elif layer_type == "relu":
            feat = relu(feat)
        elif layer_type == "maxpool2d":
            size = layer_data[1]
            feat = max_pool2d(feat, size)
        elif layer_type == "linear":
            if len(feat.shape) != 1:
                feat = np.transpose(feat, (2, 0, 1)).flatten()
            weights = layer_data[1]
            bias = layer_data[2]
            feat = linear(feat, weights, bias)
        else:
            raise NotImplementedError()
    
    return feat

In [8]:
def evaluate_deep_extractor(img, vgg16):
    '''
    Evaluates the deep feature extractor for a single image.

    [input]
    * image: numpy.ndarray of shape (H,W,3)
    * vgg16: prebuilt VGG-16 network.

    [output]
    * diff: difference between the two feature extractor's result
    '''
    
    vgg16_weights = util.get_VGG16_weights()
    img_torch = preprocess_image(img)
    
    feat = extract_deep_feature(np.transpose(img_torch.numpy(), (1,2,0)), vgg16_weights)
    
    with torch.no_grad():
        vgg_classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-3])
        vgg_feat_feat = vgg16.features(img_torch[None, ])
        vgg_feat_feat = vgg_classifier(vgg_feat_feat.flatten())
    
    return np.sum(np.abs(vgg_feat_feat.numpy() - feat))

In [13]:
path_img = "./data/aquarium/sun_aztvjgubyrgvirup.jpg"
image = io.imread(path_img)

vgg16 = torchvision.models.vgg16(pretrained=True).double()
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
print(evaluate_deep_extractor(image, vgg16))

(224, 224, 3)
0 conv2d (224, 224, 3)
1 relu (224, 224, 64)
2 conv2d (224, 224, 64)
3 relu (224, 224, 64)
4 maxpool2d (224, 224, 64)
5 conv2d (112, 112, 64)
6 relu (112, 112, 128)
7 conv2d (112, 112, 128)
8 relu (112, 112, 128)
9 maxpool2d (112, 112, 128)
10 conv2d (56, 56, 128)
11 relu (56, 56, 256)
12 conv2d (56, 56, 256)
13 relu (56, 56, 256)
14 conv2d (56, 56, 256)
15 relu (56, 56, 256)
16 maxpool2d (56, 56, 256)
17 conv2d (28, 28, 256)
18 relu (28, 28, 512)
19 conv2d (28, 28, 512)
20 relu (28, 28, 512)
21 conv2d (28, 28, 512)
22 relu (28, 28, 512)
23 maxpool2d (28, 28, 512)
24 conv2d (14, 14, 512)
25 relu (14, 14, 512)
26 conv2d (14, 14, 512)
27 relu (14, 14, 512)
28 conv2d (14, 14, 512)
29 relu (14, 14, 512)
30 maxpool2d (14, 14, 512)
31 linear (7, 7, 512)
32 relu (4096,)
33 linear (4096,)
3.1738951744575417e-12


### Building a Visual Recognition System: Revisited

We want to compare how useful deep features are in a visual recognition system. Since the speed of the function ``` scipy.ndimage.convolve``` is not ideal, you can use the pytroch VGG-16 network directly (refer to the helper function ```evaluate_deep_extractor``` on how to use the pre-trained network as feature extractor).

#### Q4.2.1 (5 Points Autograder + WriteUp):
Implement the functions
```
                    def build_recognition_system(vgg16):
```
and
```
                    def eval_recognition_system(vgg16)}
```
both of which takes the pretrained VGG-16 network as the input arguments. 

The former function should produce ``trained_system_deep.npz`` as the output. 

Included will be:
* $features$ : a $N \times  K$ matrix containing all the deep features of the $N$ training images in the data set.
* $labels$ : an $N$ vector containing the labels of each of the images. ($features[i]$ will correspond to label $labels[i]$).

The latter function should produce the confusion matrix, as with the previous question.

Instead of using the histogram intersection similarity, write a function to just use the negative Euclidean distance (as larger values are more similar).

**Report the confusion matrix and accuracy for your results in your write-up. Can you comment in your writeup on whether the results are better or worse than classical BoW - why do you think that is?**

In [3]:
def get_image_feature(args):
    '''
    Extracts deep features from the prebuilt VGG-16 network.
    This is a function run by a subprocess.
    [input]
    * i: index of training image
    * image_path: path of image file
    * vgg16: prebuilt VGG-16 network.
    
    [output]
    * feat: evaluated deep feature
    '''
    i, image_path, vgg16 = args
    img_torch = io.imread(image_path)
    print(i)
    
    '''
    HINTS:
    1.> Think along the lines of evaluate_deep_extractor
    '''
    # ----- TODO -----
    # YOUR CODE HERE
    img_torch = preprocess_image(image)
    
    with torch.no_grad():
        vgg_classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-3])
        vgg_feat_feat = vgg16.features(img_torch[None, ])
        vgg_feat_feat = vgg_classifier(vgg_feat_feat.flatten())
    feat = vgg_feat_feat.numpy()
    
    return [i,feat]

In [ ]:
def build_recognition_system(vgg16, num_workers=2):
    '''
    Creates a trained recognition system by generating training features from all training images.

    [input]
    * vgg16: prebuilt VGG-16 network.
    * num_workers: number of workers to process in parallel

    [saved]
    * features: numpy.ndarray of shape (N,K)
    * labels: numpy.ndarray of shape (N)
    '''

    train_data = np.load("./data/train_data.npz", allow_pickle=True)
    '''
    HINTS:
    1.> Similar approach as Q1.2.2 and Q3.1.1 (create an argument list and use multiprocessing)
    2.> Keep track of the order in which input is given to multiprocessing
    '''
    # YOUR CODE HERE
    image_names = train_data['files']
    image_labels = train_data['labels']
    
    # I tried multiprocessing but it would hang and I couldn't find a solution
    ordered_features = []
    for i in range(image_names.shape[0]):
        full_image_name = './data/' + image_names[i]
        i, feat = get_image_feature([i, full_image_name, vgg16])
        ordered_features.append(feat.flatten())
    

    '''
    HINTS:
    1.> reorder the features to their correct place as input
    '''
    # YOUR CODE HERE
    ordered_features = np.array(ordered_features)
    labels = np.array(image_labels)
        
    print("done", ordered_features.shape)
    
    np.savez('trained_system_deep.npz', features=ordered_features, labels=labels)

build_recognition_system(vgg16, num_workers=1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
def helper_func(args):
    # YOUR CODE HERE
    i, file_path, vgg16, trained_features, train_labels = args
    feature = get_image_feature([i, file_path, vgg16])[1]
    distances = scipy.spatial.distance.cdist(feature.reshape((1, -1)), trained_features).ravel()
    nearest_image_idx = np.argmin(distances)
    pred_label = train_labels[nearest_image_idx]
    return [i, pred_label]


def evaluate_recognition_system(vgg16, num_workers=2):
    '''
    Evaluates the recognition system for all test images and returns the confusion matrix.

    [input]
    * vgg16: prebuilt VGG-16 network.
    * num_workers: number of workers to process in parallel

    [output]
    * conf: numpy.ndarray of shape (8,8)
    * accuracy: accuracy of the evaluated system
    '''
    '''
    HINTS:
    (1) Students can write helper functions (in this cell) to use multi-processing
    '''
    test_data = np.load("./data/test_data.npz", allow_pickle=True)

    # ----- TODO -----
    trained_system = np.load("trained_system_deep.npz", allow_pickle=True)
    image_names = test_data['files']
    test_labels = test_data['labels']

    trained_features = trained_system['features']
    train_labels = trained_system['labels']

    print("Trained features shape: ", trained_features.shape)
    
    '''
    HINTS:
    1.> [Important] Can write a helper function in this cell of jupyter notebook for multiprocessing
    
    2.> Helper function will compute the vgg features for test image (get_image_feature) and find closest
        matching feature from trained_features.
    
    3.> Since trained feature is of shape (N,K) -> smartly repeat the test image feature N times (bring it to
        same shape as (N,K)). Then we can simply compute distance in a vectorized way.
    
    4.> Distance here can be sum over (a-b)**2
    
    5.> np.argmin over distance can give the closest point
    '''
    # YOUR CODE HERE
    ordered_labels = []
    for i in range(image_names.shape[0]):
        full_image_name = './data/' + image_names[i]
        i, pred_label = helper_func([i, full_image_name, vgg16, trained_features, train_labels])
        ordered_labels.append(pred_label)
    ordered_labels = np.array(ordered_labels)
        
    print("Predicted labels shape: ", ordered_labels.shape)
    
    '''
    HINTS:
    1.> Compute the confusion matrix (8x8)
    '''
    # YOUR CODE HERE
    conf_matrix = np.zeros((8, 8))
    for n in range(image_names.shape[0]):
        i = test_labels[n]
        j = ordered_labels[n]
        conf_matrix[i, j] += 1
    accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
    
    np.save("./trained_conf_matrix.npy",conf_matrix)
    return conf_matrix, accuracy
    # pass
    
evaluate_recognition_system(vgg16, num_workers=7)